# BIKE - Bit Flipping Key Encapsulation

- Criação de um protótipo em Sagemath para o algoritmo **BIKE**.
- Pretende-se implementar um **KEM**, que seja **IND-CPA** seguro, e um **PKE** que seja **IND-CCA** seguro.

## Protótipo

In [1]:
from sage.all import *

### Parâmetros

Parâmetros para o nível de segurança 1

In [2]:
r = 257  # 12323  # Comprimento do bloco (block length)
n = r * 2  # Comprimento do código (code length)
w = 142  # Peso da linha (row weight)
t = 16 #134  # Peso do erro (error weight)
l = 256  # Comprimento do segredo partilhado (shared secret size) | NOTA: Este parametro é fixo para todos os níveis de segurança

# BGF decoder parameters - nível de segurança 1
NbIter = 5  # Número de iterações do decoder
tau = 3  # Threshold Gap | TODO: Confirmar se este comentário está correto
threshold = lambda S, _i: max(floor(0.0069722 * S + 13.530), 36)  # Threshold function

In [3]:
F = GF(2)

M = F ** l  # Message space

R = PolynomialRing(F, 'x')
x = R.gen()
Rr = QuotientRing(R, R.ideal(x ** r - 1))  # Polynomial ring R / (x^r - 1)

KK = F ** l  # Private key space

print("Message space M:   ", M)
print("Shared key space K:", KK)
print("Polynomial ring R: ", R)
print("Quotient ring Rr:  ", Rr)

MElement = type(M.random_element())  # Basicamente binário
RElement = type(Rr.random_element())  # Elemento de Rr
KElement = type(KK.random_element())  # Basicamente binário

print("MElement:", MElement)
print("RElement:", RElement)
print("KElement:", KElement)

Message space M:    Vector space of dimension 256 over Finite Field of size 2
Shared key space K: Vector space of dimension 256 over Finite Field of size 2
Polynomial ring R:  Univariate Polynomial Ring in x over Finite Field of size 2 (using GF2X)
Quotient ring Rr:   Univariate Quotient Polynomial Ring in xbar over Finite Field of size 2 with modulus x^257 + 1
MElement: <class 'sage.modules.vector_mod2_dense.Vector_mod2_dense'>
RElement: <class 'sage.rings.polynomial.polynomial_quotient_ring.PolynomialQuotientRing_generic_with_category.element_class'>
KElement: <class 'sage.modules.vector_mod2_dense.Vector_mod2_dense'>


### Funções auxiliares

In [4]:
def generate_sparse(weight: int, size: int) -> RElement:
    """
    Gera um sparse vector.
    Entrada: weight - número de elementos não nulos (Hamming weight)
             size - tamanho do vector
    Saída: elemento de Rr
    """
    while True:
        # Generate a random list of size 'size' with 'weight' non-zero elements
        sparse_rep = [0] * size
        for _ in range(weight):
            rand_index = randint(0, size - 1)
            while sparse_rep[rand_index] != 0:
                rand_index = randint(0, size - 1)

            sparse_rep[rand_index] = 1

        assert sum(sparse_rep) == weight
        return Rr(sparse_rep)

In [5]:
def bytes_to_bits(b: bytes) -> list:
    assert type(b) == bytes

    return [int(bit) for byte in b for bit in bin(byte)[2:].zfill(8)]

In [6]:
def expand(lis: list, size: int) -> list:
    assert type(lis) == list

    return lis + [0] * (l - len(lis))

In [7]:
# noinspection PyPep8Naming
def R_to_bytes(r: RElement) -> bytes:
    assert type(r) == RElement

    return bytes(r.list())


# noinspection PyPep8Naming
def bytes_to_R(b: bytes) -> RElement:
    assert type(b) == bytes

    return Rr(list(b))


assert bytes_to_R(R_to_bytes(Rr([1, 0, 1]))) == Rr([1, 0, 1])


# noinspection PyPep8Naming
def M_to_bytes(m: MElement) -> bytes:
    assert type(m) == MElement

    bits = m.list()
    bit_string = ''.join(str(bit) for bit in bits)  # convert the list of bits to a string

    return int(bit_string, 2).to_bytes(len(bits) // 8, byteorder='big')


# noinspection PyPep8Naming
def bytes_to_M(b: bytes) -> MElement:
    assert type(b) == bytes

    bytess = expand(bytes_to_bits(b), l)

    assert len(bytess) == l

    return M(bytess)

assert bytes_to_M(M_to_bytes(M([1, 0] * (l // 2)))) == M([1, 0] * (l // 2))

In [8]:
def getHammingWeight(m: MElement) -> int:
    acc = 0
    for i in m:
        if i == 1:
            acc += 1

    return acc


assert getHammingWeight(M([1, 0] * (l // 2))) == l // 2

In [9]:
def xor(a: MElement, b: MElement) -> MElement:
    assert len(a) == len(b)
    return M([a[i] ^ b[i] for i in range(len(a))])

### Funções de Hash necessárias

#### Função H

In [10]:
# noinspection PyPep8Naming
def H(m: bytes) -> (RElement, RElement):
    # TODO: Migrate this to use AES256-CTR PRNG

    e0 = generate_sparse(t, r)
    e1 = generate_sparse(t, r)

    return e0, e1


H(b'')

(xbar^248 + xbar^243 + xbar^229 + xbar^220 + xbar^204 + xbar^200 + xbar^192 + xbar^154 + xbar^130 + xbar^118 + xbar^100 + xbar^90 + xbar^88 + xbar^82 + xbar^80 + xbar^16,
 xbar^217 + xbar^208 + xbar^181 + xbar^164 + xbar^154 + xbar^112 + xbar^94 + xbar^88 + xbar^75 + xbar^69 + xbar^64 + xbar^51 + xbar^37 + xbar^30 + xbar^24 + xbar^6)

#### Função L

In [11]:
# noinspection PyPep8Naming
def L(e0: RElement, e1: RElement) -> MElement:
    # Apply the SHA384 hash function to the concatenation of e0 and e1
    from hashlib import sha384

    digest = sha384(R_to_bytes(e0) + R_to_bytes(e1)).digest()

    # Concat all the bits of the digest into a list of bits
    digest = bytes_to_bits(digest[:l // 8])  # We only need l bits (l / 8 bytes)

    return M(digest)  # Returns the MElement corresponding to the digest


L(Rr([1, 0, 1]), Rr([1, 0, 1]))

(0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1)

#### Função K

In [12]:
# noinspection PyPep8Naming
def K(m: MElement, c0: RElement, c1: MElement) -> KElement:
    # Apply the SHA384 hash function to the concatenation of m, c0 and c1
    from hashlib import sha384

    digest = sha384(M_to_bytes(m) + R_to_bytes(c0) + M_to_bytes(c1)).digest()

    digest = bytes_to_bits(digest[:l // 8])  # We only need l bits (l / 8 bytes)

    return KK(digest)  # Returns the KElement corresponding to the digest


K(M([1, 0] * 128), Rr([1, 0, 1]), M([1, 0] * 128))

(1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0)

### Função de computação do sindrome (syndrome computation)

In [13]:
def compute_syndrome(c0: RElement, h0: RElement) -> RElement:
    return c0 * h0

### Geração de chaves

In [14]:

def keygen() -> ((RElement, RElement), MElement, RElement):
    """
    Geração de chaves
    Entrada: Nenhum
    Saída: (pk, sk)
    """
    h0 = generate_sparse(w // 2, l)
    h1 = generate_sparse(w // 2, l)

    sigma = os.urandom(32)  # FIXME: Estou a gerar o sigma assim por agora.

    h0_inv = 1 / h0
    h = h1 * h0_inv

    return (h0, h1), sigma, h

In [15]:
# Teste TODO: Mover isto para a secção de testes

(priv_key, sigma, public_key) = keygen()

# Print the hex representation of the public key and secret key
print("public_key: ", public_key.lift())

public_key:  x^255 + x^253 + x^252 + x^249 + x^245 + x^244 + x^240 + x^239 + x^237 + x^236 + x^234 + x^233 + x^231 + x^224 + x^223 + x^220 + x^218 + x^217 + x^215 + x^214 + x^211 + x^210 + x^209 + x^206 + x^205 + x^202 + x^201 + x^198 + x^197 + x^196 + x^194 + x^192 + x^190 + x^189 + x^188 + x^186 + x^184 + x^182 + x^179 + x^177 + x^176 + x^171 + x^170 + x^169 + x^168 + x^165 + x^163 + x^162 + x^161 + x^160 + x^158 + x^157 + x^156 + x^154 + x^153 + x^152 + x^151 + x^146 + x^142 + x^141 + x^137 + x^134 + x^133 + x^131 + x^130 + x^128 + x^127 + x^125 + x^124 + x^123 + x^119 + x^118 + x^117 + x^113 + x^108 + x^106 + x^104 + x^101 + x^100 + x^97 + x^95 + x^93 + x^91 + x^89 + x^86 + x^85 + x^84 + x^82 + x^77 + x^76 + x^75 + x^74 + x^72 + x^68 + x^63 + x^61 + x^60 + x^59 + x^56 + x^55 + x^51 + x^50 + x^44 + x^37 + x^36 + x^35 + x^34 + x^33 + x^32 + x^31 + x^29 + x^27 + x^26 + x^25 + x^23 + x^20 + x^16 + x^14 + x^13 + x^12 + x^10 + x^9 + x^8 + x^6 + x^5


### Encapsulamento

In [16]:
def calculate_c(e0: RElement, e1: RElement, h: RElement, seed: MElement) -> (RElement, MElement):
    return e0 + e1 * h, seed + L(e0, e1)

In [17]:
def encapsulate(h: RElement) -> (KElement, (RElement, MElement)):
    seed: MElement = M.random_element()
    (e0, e1) = H(seed)

    c = calculate_c(e0, e1, h, seed)
    c0, c1 = c

    k = K(seed, c0, c1)

    return k, c

In [18]:
# Teste TODO: Mover isto para a secção de testes

(priv_key, sigma, public_key) = keygen()

(k, c) = encapsulate(public_key)

print("k: ", k.lift())
print("c: ", c)

k:  (1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1)
c:  (xbar^254 + xbar^253 + xbar^252 + xbar^251 + xbar^250 + xbar^249 + xbar^248 + xbar^247 + xbar^246 + xbar^245 + xbar^244 + xbar^243 + xbar^242 + xbar^239 + xbar^235 + xbar^232 + xbar^228 + xbar^221 + xbar^220 + xbar^219 + xb

### Desencapsulamento

In [24]:
# noinspection PyUnresolvedReferences
from sage.matrix.matrix_integer_dense import Matrix_integer_dense


def decoder(x: RElement, h0: RElement, h1: RElement) -> (RElement, RElement):
    print("decoder function")

    # Convert x to a vectorSpace element
    x = RElement_to_VectorSpace(x)


    H_mat = get_H_matrix(h0, h1)

    BGF(x, H_mat)



def BGF(s: RElement, H: Matrix_integer_dense) -> (RElement, RElement):
    print("BGF function")
    e = copy(VectorSpace(GF(2), n).zero())
    d = w // 2

    HTranspose = H.transpose()

    for i in range(1, NbIter + 1):
        T = threshold(getHammingWeight(s + e * HTranspose), i)
        #e, black, gray = BFIter(s + e * HTranspose, e, T, H)
        #if i == 1:
        #    e = BFMaskedIter(s + e * HTranspose, e, black, ((d + 1) // 2) + 1, H)
        #    e = BFMaskedIter(s + e * HTranspose, e, gray, ((d + 1) // 2) + 1, H)

    if s == e * HTranspose:
        (e0, e1) = e[:r], e[r:]
        return e0, e1
    else:
        print("Decoding failed!")
        return None, None

def BFIter(s: RElement, e: RElement, T: int, H: Matrix_integer_dense) -> (RElement, RElement, RElement):
    """
    Black-Gray-Flip (BGF) BFIter function.
    :param s: the syndrome vector
    :param e: the error vector
    :param T: the threshold
    :param H: the parity-check matrix
    :return: a tuple containing the updated error vector, the set of black bits, and the set of gray bits
    """
    n = H.ncols()
    black = copy(VectorSpace(GF(2), n).zero())
    gray = copy(VectorSpace(GF(2), n).zero())

    for j in range(n):
        if ctr(H, s, j) >= T:
            e[j] += 1  # TODO: Check if this is correct
            black[j] = 1
        elif ctr(H, s, j) >= T - tau:
            gray[j] = 1

    return e, black, gray


def ctr(H: Matrix_integer_dense, s: RElement, j: int) -> int:
    """
    ctr(H; s; j). This function computes a quantity referred to as the counter (aka the number of unsatisfied parity-checks) of j.
    It is the number of ’1’ (set bits) that appear in the same position in the syndrome s and in the j-th column of the matrix H.
    """
    return getHammingWeight(s.pairwise_product(H.column(j)))


def BFMaskedIter(s: RElement, e: RElement, mask: RElement, T: int, H: Matrix_integer_dense) -> RElement:
    """
    Black-Gray-Flip (BGF) BFMaskedIter function.
    :param s: the syndrome vector
    :param e: the error vector
    :param mask: the mask vector
    :param T: the threshold
    :param H: the parity-check matrix
    :return: the updated error vector
    """
    n = H.ncols()

    for j in range(n):
        if ctr(H, s, j) >= T:
            e[j] += 1

    return e

In [25]:
def RElement_to_VectorSpace(element: RElement) -> type(VectorSpace(GF(2), n)):
    print("RElement_to_VectorSpace function")

    elem_coefs = element.lift().list()

    v = vector(GF(2), elem_coefs + [0] * (r - len(elem_coefs)))

    return v


In [26]:
def get_H_matrix(h0: RElement, h1: RElement) -> Matrix_integer_dense:
    print("get_H_matrix function")
    #print("H0: ", len(get_circulant_matrix(h0)[0]))
    #print("H1: ", len(get_circulant_matrix(h1)[0]))
    #print("H: ", block_matrix(1, 2, [get_circulant_matrix(h0), get_circulant_matrix(h1)]))

    H = block_matrix(1, 2, [get_circulant_matrix(h0), get_circulant_matrix(h1)])

    print("H dimensions: ", H.dimensions())
    assert H.dimensions() == (r, n)

    return H


# noinspection PyUnresolvedReferences
def get_circulant_matrix(element: RElement) -> Matrix_integer_dense:
    print("get_circulant_matrix function")
    vec = element.lift().list()
    # Fill the rest of the vector with zeros
    vec = vec + [0] * (r - len(vec))

    circ = matrix.circulant(vec)
    #print("circ: ", circ)

    return circ

#get_circulant_matrix(Rr([1, 0, 1])).augment(get_circulant_matrix(Rr([0, 0, 1])))
#block_matrix(1, 2, [get_circulant_matrix(Rr([1, 0, 1])), get_circulant_matrix(Rr([0, 0, 1]))])

In [29]:
def decapsulate(h0: RElement, h1: RElement, sigma: bytes, c0: RElement, c1: MElement) -> KElement:
    e_ = decoder(c0 * h0, h0, h1)

    print("e_: ", e_)

    if e_ == (None, None):
        return None

    m_ = c1 + L(e_[0], e_[1])

    if e_ == H(m_):
        return K(m_, c0, c1)
    else:
        return K(sigma, c0, c1)

In [30]:
# Teste TODO: Mover isto para a secção de testes

(priv_key, sigma, public_key) = keygen()

(k, (c0, c1)) = encapsulate(public_key)

k_ = decapsulate(*priv_key, sigma, c0, c1)

decoder function
RElement_to_VectorSpace function
get_H_matrix function
get_circulant_matrix function
get_circulant_matrix function
H dimensions:  (257, 514)
BGF function
Decoding failed!
e_:  None


TypeError: 'NoneType' object is not subscriptable

## KEM (Key Encapsulation Mechanism) - IND-CPA (INDistinguishable under Chosen Plaintext Attack)

## PKE (Public Key Encryption) - IND-CCA (INDistinguishable under Chosen Ciphertext Attack)

In [104]:
def encrypt(h: RElement, m: MElement) -> (RElement, MElement):
    e0 = generate_sparse(t, r)
    e1 = generate_sparse(t, r)

    c0 = e0 + e1 * h
    c1 = m + L(e0, e1)

    return c0, c1

In [108]:
def decrypt(h0: RElement, h1: RElement, s: RElement) -> (RElement, RElement):
    return decoder(s * h0, h0, h1)

In [109]:
(priv_key, sigma, public_key) = keygen()

message = "Hello World!"
messageM = bytes_to_M(bytes(message, 'utf-8'))

(c0, c1) = encrypt(public_key, messageM)

m_ = decrypt(priv_key, c0, c1)

256


TypeError: 'sage.modules.vector_mod2_dense.Vector_mod2_dense' object cannot be interpreted as an integer